<a href="https://colab.research.google.com/github/Athom77/INVEST_IO/blob/master/stock_scraping_investing_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from tqdm import tqdm_notebook as tqdm
import pandas
import json
import pprint

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

     |████████████████████████████████| 911kB 2.8MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubu

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: use options instead of chrome_options


In [ ]:
#CALENDARIO ECONOMICO
#eventRowId_410946 > td.left.textNum.sentiment.noWrap
#wd.get('https://www.investing.com/economic-calendar/')

url = 'https://www.investing.com/economic-calendar'
import time
import numpy as np

from selenium import webdriver
wd.get(url)

for tr in wd.find_elements_by_xpath('//*[@id="economicCalendarData"]/tbody//tr'):
    tds = tr.find_elements_by_tag_name('td')
    print ([td.text for td in tds])
    time.sleep(2+np.random.randn()*0.2)

#Extract Message for each user on a page //*[@id="eventRowId_410946"]/td[3]
#volatility = wd.find_element_by_xpath('//*[@id="eventRowId_410946"]/td[3]')[0]
#volatility_lvl = volatility.text
#print(volatility_lvl)

['Sunday, July 26, 2020']
['19:50', '  JPY', '', 'BoJ Summary of Opinions  ', ' ', ' ', ' ', '']
['19:50', '  JPY', '', 'Capital Spending (YoY)', ' ', '4.3%', '-3.5%', '']
['20:00', '  AUD', '', 'RBA Assist Gov Kent Speaks  ', ' ', ' ', ' ', '']
['21:30', '  CNY', '', 'Chinese Industrial profit (YoY) (Jun)', ' ', ' ', '6.00%', '']
['21:30', '  CNY', '', 'Chinese Industrial profit YTD (Jun)', ' ', ' ', '-19.3%', '']


In [ ]:
import time
url = 'https://www.investing.com/economic-calendar'
time.sleep(2+np.random.randn()*0.2)



wd.get(url)
print(len(wd.find_elements_by_xpath('//*[@id="economicCalendarData"]/tbody/tr')))

In [ ]:
import time
url = 'https://www.investing.com/economic-calendar'
time.sleep(2+np.random.randn()*0.2)

# click submit button CALENDARIO ACCEDI
#submit_button = wd.find_elements_by_xpath('//*[@id="timeFrame_nextWeek"]/input')[0]
#submit_button.click()

# click submit button CALENDARIO ACCEDI
submit_button = wd.find_elements_by_xpath('//div[@id="economicCalendarFilters"]/div/a[6]')
submit_button.click()

# type text DATA INIZIO
text_area = wd.find_element_by_css_selector('#startdate')
text_area.send_keys("01/01/2020")
time.sleep(2+np.random.randn()*0.2)

# type text DATA FINE
text_area = wd.find_element_by_css_selector('#enddate')
text_area.send_keys("01/01/2021")

#applyBtn -APPLY

submit_button = wd.find_elements_by_css_selector('#applyBtn')
submit_button.click()
time.sleep(2+np.random.randn()*0.2)

wd.get(url)
print(len(wd.find_elements_by_xpath('//*[@id="economicCalendarData"]/tbody/tr')))

AttributeError: ignored

In [ ]:
import time
url = 'https://www.investing.com/economic-calendar'
time.sleep(2+np.random.randn()*0.2)

# click submit button CALENDARIO ACCEDI
submit_button = wd.find_element_by_xpath('//*[@id="timeFrame_nextWeek"]')
submit_button.click()

wd.get(url)
print(len(wd.find_elements_by_xpath('//*[@id="economicCalendarData"]/tbody/tr')))

NoSuchElementException: ignored